In [19]:
import numpy as np
import pandas as pd
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM,  Dropout
from keras.preprocessing.text import Tokenizer
##from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

In [3]:
dataset = pd.read_csv("sentiment_data.csv")

In [4]:
df = dataset.copy()
df.head()

,Unnamed: 0,Rating,Review
0,0,1.0,3 yıldır tık demedi. :)
1,1,1.0,3 yıldır kullanıyorum müthiş
2,2,1.0,Ürün bugün elime geçti çok fazla inceleme fırs...
3,3,1.0,Almaya karar verdim. Hemencecik geldi. Keyifle...
4,4,1.0,Günlük kullanımınızı çok çok iyi karsılıyor kı...


In [5]:
df = df.drop(['Unnamed: 0'], axis=1)
df = df.dropna()
df

,Rating,Review
0,1.0,3 yıldır tık demedi. :)
1,1.0,3 yıldır kullanıyorum müthiş
2,1.0,Ürün bugün elime geçti çok fazla inceleme fırs...
3,1.0,Almaya karar verdim. Hemencecik geldi. Keyifle...
4,1.0,Günlük kullanımınızı çok çok iyi karsılıyor kı...
...,...,...
281937,0.0,turkcell'den 2000 yılında new york borsasında ...
281938,0.0,Yeni telefon sapığım Turkcell .Açmadığım halde...
281939,0.0,106 lira fatura mı olur turkcell sadece soruyorum
281940,0.0,@Buragl111 turkcell kazık takıl sen bimcell le...


In [6]:
df['Rating'] = df['Rating'].astype('int64')
df

,Rating,Review
0,1,3 yıldır tık demedi. :)
1,1,3 yıldır kullanıyorum müthiş
2,1,Ürün bugün elime geçti çok fazla inceleme fırs...
3,1,Almaya karar verdim. Hemencecik geldi. Keyifle...
4,1,Günlük kullanımınızı çok çok iyi karsılıyor kı...
...,...,...
281937,0,turkcell'den 2000 yılında new york borsasında ...
281938,0,Yeni telefon sapığım Turkcell .Açmadığım halde...
281939,0,106 lira fatura mı olur turkcell sadece soruyorum
281940,0,@Buragl111 turkcell kazık takıl sen bimcell le...


In [7]:
df['Rating'].unique().tolist()


[1, 0]

In [8]:
target = df['Rating'].values.tolist()#negatif=0, pozitif=1
data = df['Review'].values.tolist()#text verisi

In [9]:
seperation = int(len(data) * 0.80)
x_train, x_test = data[:seperation], data[seperation:]
y_train, y_test = target[:seperation], target[seperation:]

In [10]:
df.shape

(281459, 2)

In [11]:
num_words = 10000
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(data)

In [12]:
import pickle

with open('turkish_tokenizer_hack.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [13]:
with open('turkish_tokenizer_hack.pickle', 'rb') as handle:
    turkish_tokenizer = pickle.load(handle)

In [14]:
x_train_tokens = turkish_tokenizer.texts_to_sequences(x_train)
x_train[100]

'Bu fiyata bu kalite kaçırmayın derim '

In [15]:
x_train_tokens[100]

[5, 44, 5, 143, 349, 148]

In [16]:
x_test_tokens = turkish_tokenizer.texts_to_sequences(x_test)

In [17]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)
num_tokens.shape

(281459,)

In [18]:
max_tokens = np.mean(num_tokens) + 2*np.std(num_tokens) 
max_tokens = int(max_tokens)
max_tokens

62

In [20]:
x_train_pad = tf.keras.utils.pad_sequences(x_train_tokens, maxlen=max_tokens)
x_test_pad = tf.keras.utils.pad_sequences(x_test_tokens, maxlen=max_tokens)
print(x_train_pad.shape)
print(x_test_pad.shape)

(225167, 62)
(56292, 62)


In [21]:
model = Sequential()
embedding_size = 50
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='embedding_layer'))

model.add(LSTM(units=16, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=8, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=4, return_sequences=False))
model.add(Dropout(0.2))
# Dense layer: Tek nörondan oluşuyor
model.add(Dense(1, activation='sigmoid'))

In [22]:
from keras.optimizers import Adam
optimizer = Adam(lr=1e-3)

C:\Users\utku.dogan\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [23]:
model.compile(loss='binary_crossentropy',
optimizer='adam',
metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_layer (Embedding)  (None, 62, 50)           500000    
                                                                 
 lstm (LSTM)                 (None, 62, 16)            4288      
                                                                 
 dropout (Dropout)           (None, 62, 16)            0         
                                                                 
 lstm_1 (LSTM)               (None, 62, 8)             800       
                                                                 
 dropout_1 (Dropout)         (None, 62, 8)             0         
                                                                 
 lstm_2 (LSTM)               (None, 4)                 208       
                                                                 
 dropout_2 (Dropout)         (None, 4)                 0

In [24]:
type(y_train)
x = np.array(y_train)
print(np.unique(x))

[0 1]


In [25]:
x_train_pad = np.array(x_train_pad)
y_train = np.array(y_train)

In [27]:
model.fit(x_train_pad, y_train, epochs=10, batch_size=256)

Epoch 1/10
880/880 [==============================] - 74s 85ms/step - loss: 0.1993 - accuracy: 0.9490
Epoch 2/10
880/880 [==============================] - 74s 85ms/step - loss: 0.1116 - accuracy: 0.9651
Epoch 3/10
880/880 [==============================] - 74s 84ms/step - loss: 0.0858 - accuracy: 0.9734
Epoch 4/10
880/880 [==============================] - 74s 84ms/step - loss: 0.0698 - accuracy: 0.9790
Epoch 5/10
880/880 [==============================] - 74s 84ms/step - loss: 0.0572 - accuracy: 0.9832
Epoch 6/10
880/880 [==============================] - 74s 84ms/step - loss: 0.0486 - accuracy: 0.9862
Epoch 7/10
880/880 [==============================] - 74s 84ms/step - loss: 0.0416 - accuracy: 0.9891
Epoch 8/10
880/880 [==============================] - 74s 85ms/step - loss: 0.0376 - accuracy: 0.9903
Epoch 9/10
880/880 [==============================] - 75s 85ms/step - loss: 0.0326 - accuracy: 0.9919
Epoch 10/10
880/880 [==============================] - 75s 85ms/step - loss: 0.028

In [28]:
x_test_pad = np.array(x_test_pad)
y_test = np.array(y_test)
result = model.evaluate(x_test_pad, y_test)
result

1760/1760 [==============================] - 29s 16ms/step - loss: 1.0614 - accuracy: 0.7806


[1.0613890886306763, 0.7805549502372742]

In [29]:
from keras.models import load_model

model.save('hack_model.h5')